# Getting started with Amazon Braket Hybrid Job
This tutorial shows how to run an Amazon Braket Hybrid Job with the example of executing multiple small circuits. There are two pieces of code required for every Braket Job, an orchestration script and an algorithm script. The orchestration script will be shown below. The algorithm script can be a seperate file or a python module. To quickly get started, we consider simple circuits with only one qubit and one gate in this example notebook.

## Algorithm script
The algorithm script for this notebook is [here](algorithm_script.py) as a separate file. The code block below is a copy of the algorithm script. As shown, each of our circuits has only one X rotation gate with a random angle. The circuit is repeated five times with different random rotations. We would write the algorithm script as we usually do, except that we do not specify the backend QPUs or simulators explicitly. Instead, it is provided through environment variables which is defined in the orchestration script shown later. In addition, we also need to specify the variables to be recorded. In this notebook, we record the measurement counts and the random angle for each random circuit. 

In [1]:
import os
import numpy as np

from braket.aws import AwsDevice
from braket.circuits import Circuit
from braket.jobs import save_job_result


print("Test job started!!!!!")

# Use the device declared in the Orchestration Script
device = AwsDevice(os.environ["AMZN_BRAKET_DEVICE_ARN"])

counts_list = []
angle_list = []
for _ in range(5):
    angle = np.pi * np.random.randn()
    random_curcuit = Circuit().rx(0, angle)

    task = device.run(random_curcuit, shots=100)
    counts = task.result().measurement_counts

    angle_list.append(angle)
    counts_list.append(counts)
    print(counts)

# Save the variables of interest so that we can access later
save_job_result({"counts": counts_list, "angle": angle_list})

print("Test job completed!!!!!")

## Orchestration script
In the orchestration script, we use <code>AwsQuantumJob</code> to create a Braket job. When the Braket job is created, it starts an AWS instance and spins up a container. The instance type, container and other configurations can be specified via keyword arguments. See the [developer guide](https://docs.aws.amazon.com/braket/latest/developerguide/what-is-braket.html) and other example notebooks for more information.

The inputs of <code>AwsQuantumJob</code> for this notebook are:
- <b>device_arn</b>: The arn of the Braket simulator or QPU we want to use. It will be stored as an environment variable for the algorithm script.
- <b>source_module</b>: The path to a file or a python module that contains your algorithm script. It will be uploaded to the container for Braket Job execution.
- <b>entry point</b>: The path relative to the source_module. It points to the piece of code to be executed when the Braket Job starts.
- <b>wait_until_complete</b>: When it is true, the program will wait until the Braket Job is completed. Otherwise, it will run asynchronously. 

In [2]:
from braket.aws import AwsQuantumJob

In [ ]:
job = AwsQuantumJob.create(
    "arn:aws:braket:::device/quantum-simulator/amazon/sv1",
    source_module="algorithm_script.py",
    entry_point="algorithm_script",
    wait_until_complete=True,
)

In this exercise, the algorithm script is in one file, so we set <code>source_module</code> to be <code>algorithm_script.py</code> and <code>entry_point</code> to be <code>algorithm_script</code>. Depend on your application, there are other options you can arrange your algorithm scripts. For example, if you wish to only execute a part of <code>algorithm_script.py</code> at the start of a Braket Job, you can package that part to be a <code>starting_function()</code>. Then the input arguments would be

In [4]:
source_module="algorithm_script.py"
entry_point="algorithm_script:starting_function"

When your algorithm script requires others files such as helper functions, you can put them all in one folder, say the <code>algorithm_folder</code>. The input arguments would then be

In [5]:
source_module="algorithm_folder"
entry_point="algorithm_folder.algorithm_script:starting_function"

## Local Job
Braket Job also provides an option to test the job locally by using LocalQuantumJob. This may save resources for setting up and debugging your Braket Job. Note that this feature requires Docker. Amazon Braket Notebook Instance has Docker pre-stalled. But if we execute a local job at a local machine, such a laptop or a desktop, we need to have Docker installed first.

In [ ]:
from braket.jobs.local.local_job import LocalQuantumJob

job = LocalQuantumJob.create(
    "arn:aws:braket:::device/quantum-simulator/amazon/sv1",
    source_module="algorithm_script.py",
    entry_point="algorithm_script",
    wait_until_complete=True,
)

## Results
If you run the Braket Job asynchronously, the status can be checked by <code>job.state()</code>. Once completed, the results are stored in Braket Job and can be retrieved by <code>job.result()</code>. Logs and metadata are also stored in Braket Job. If you lose the job variable, you can always retrieve it by its arn which you can find in the console.

In [7]:
results = job.result()
print("counts")
print(results["counts"])
print("angles")
print(results["angles"])

counts
[{'1': 79, '0': 21}, {'1': 94, '0': 6}, {'1': 94, '0': 6}, {'0': 99, '1': 1}, {'0': 66, '1': 34}]
angles
[2.07419029465829, 2.7510018628135975, 2.6861075414832465, 0.41560307276445757, 1.0596381021855892]


In [8]:
%%capture captured
print(job.logs())
print(job.metadata())

The job result can also be downloaded to the current directory by

In [9]:
job.download_result()

## Summary
In this tutorial, we have prepared an algorithm and an orchestration script for a simple example. We have learned how to create a Braket Job and how to retrieve the results once the job is completed.